# Predicción de tendencia del dolar

Este Jupyter Notebook tiene como objetivo desarrollar y evaluar modelos predictivos para la tendencia del dólar utilizando dos algoritmos de aprendizaje automático: Random Forest y Gradient Boosting. Para ello, se implementarán dos pipelines de procesamiento y modelado distintos, cada uno asociado a uno de estos algoritmos. El análisis se realizará sobre un conjunto de datos históricos de las tasas de cambio del dólar, incluyendo diversas variables que podrían influir en su comportamiento.

## Instalamos paquetes

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, ConfusionMatrixDisplay, confusion_matrix, mean_absolute_error
from sklearn.decomposition import PCA

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (
    MinMaxScaler,
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
    PolynomialFeatures,
    FunctionTransformer,
)

Hacemos esas cosas genéricas que el profe siempre hace en clase

In [2]:
# Definamos el "random_state" para que los resultados sean reproducibles:
random_state=42
# Cambiemos la fuente de las gráficas de matplotlib:
plt.rc('font', family='serif', size=12)

## Descargamos los datos

Nos traemos el csv y creamos el dataset. El head() lo estamos usando para asegurarnos que tenemos los datos que son.

In [3]:
!wget --no-check-certificate 'https://docs.google.com/spreadsheets/d/1N90hH0WpO7IhK5ptAW-kWakol71F8GfkiFp0bYlyMVI/gviz/tq?tqx=out:csv&sheet=ML-Dolar%20(Full%20Dataset)' -O dollar_dataset.csv


--2023-10-23 17:46:21--  https://docs.google.com/spreadsheets/d/1N90hH0WpO7IhK5ptAW-kWakol71F8GfkiFp0bYlyMVI/gviz/tq?tqx=out:csv&sheet=ML-Dolar%20(Full%20Dataset)
Resolving docs.google.com (docs.google.com)... 142.250.101.138, 142.250.101.100, 142.250.101.139, ...
Connecting to docs.google.com (docs.google.com)|142.250.101.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘dollar_dataset.csv’

dollar_dataset.csv      [ <=>                ]  66.65K  --.-KB/s    in 0.005s  

2023-10-23 17:46:21 (13.0 MB/s) - ‘dollar_dataset.csv’ saved [68249]



In [4]:
# Cargar los datos
dollar_df = pd.read_csv("dollar_dataset.csv", index_col=0)
dollar_df.dtypes

# Este drop elimina las columnas no influyentes. No lo hacemos en este notebook porque queremos gráficos que nos permitan conclusiones
# dollar_df.drop(columns=['inflacion_eu', 'pib_pesos', 'deuda_ext_total', 'inflacion_usa', 'deuda_ext_publica'], inplace=True)

inflacion_col                    float64
inflacion_usa                    float64
inflacion_eu                     float64
deuda_ext_publica                float64
deuda_ext_privada                float64
deuda_ext_total                  float64
pib_pesos                        float64
prime_rate                       float64
cafe_miles_dolares               float64
cafe_toneladas_cubicas           float64
carbon_miles_dolares             float64
carbon_toneladas_cubicas         float64
petroleo_miles_dolares           float64
petroleo_toneladas_cubicas       float64
ferroniquel_miles_dolares        float64
ferroniquel_toneladas_cubicas    float64
trm                              float64
dtype: object

## Vamos a partir el dataset en datos de entrenamiento y datos de prueba

Como no tenemos suficientes datos, no vamos a trabajar con datos de validación

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    dollar_df.drop(columns='trm'),
    dollar_df['trm'],
    test_size = 0.2,
    random_state = random_state
)

## Nos embarcamos en la travesía de quitar los nulos y escalar los datos

Debido a que los datos que tenemos son menusales, pero temas de inflación y deuda externa los tenemos anuales, debemos normalizar esas columnas para que tengamos datos mensuales que puedan contribuir a la predicción.

Vamos a crear el pipeline que necesitamos.

In [ ]:
dollar_df.hist(bins=50, figsize=(20, 15));

In [ ]:
X_train.describe()


In [ ]:
# Listamos la cantidad de columnas con cantidad de nulos
X_train.isna().sum()

## Vamos a arrancar nuestro proceso para entrenar el modelo

In [ ]:
# Encontrar variables numéricas y categóricas
num_cols = X_train.select_dtypes(include=np.number).columns

# Definir el pipeline de pre-procesamiento
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")),("scaler", StandardScaler())]
)

preprocessor = ColumnTransformer(
    transformers = [
       ('num',numeric_transformer,num_cols)
      ])

#Definimos nuestro regresor
rf_base = RandomForestRegressor(random_state=random_state)
gb_base = GradientBoostingRegressor(random_state=random_state)

#Definimos los pipelines
pipeline_rf = Pipeline(steps = [
    ('preprocessor', preprocessor),
    ('regressor', rf_base),
                            ])
pipeline_gb = Pipeline(steps = [
    ('preprocessor', preprocessor),
    ('regressor', gb_base),
    ])
#{'regressor__max_depth': 9, 'regressor__min_samples_leaf': 10, 'regressor__n_estimators': 100}
# Definamos la malla de parámetros sobre la que haremos la búsqueda:
param_grid = {
    'regressor__n_estimators': [50, 100],
    'regressor__max_depth': [9],
    'regressor__min_samples_leaf': [10, 300, 2000]
}

# Definamos nuestros modelo mediante GridSearchCV:
rf = GridSearchCV(pipeline_rf, cv=3, param_grid=param_grid)
gb = GridSearchCV(pipeline_gb, cv=3, param_grid=param_grid)

In [ ]:
%%time
# Entrenemos el random forest:
rf.fit(X_train, y_train)
# Entrenemos el gradient boosting:
gb.fit(X_train, y_train)

In [ ]:
print(rf.best_params_)
print(gb.best_params_)


In [ ]:
# Obtengamos el R^2 y el MAE de entrenamiento para cada modelo:
print("Train set")
for model in (rf, gb):
    print(f"Model: {'Random Forest' if model == rf else 'Gradient Boosting'}")
    print(f'R^2: {model.score(X_train, y_train)}')
    print(f'MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
    print('\n')

print("Test set")
for model in (rf, gb):
    print(f"Model: {'Random Forest' if model == rf else 'Gradient Boosting'}")
    print(f'R^2: {model.score(X_test, y_test)}')
    print(f'MAE: {mean_absolute_error(y_test, model.predict(X_test))}')
    print('\n')


In [ ]:
# Veamos los datos junto con las predicciones:
pd.concat([
    dollar_df.drop(columns='trm').reset_index(drop=True),
    dollar_df['trm'].reset_index(drop=True),
    pd.DataFrame({'rf_predicted_trm': rf.predict(dollar_df.drop(columns=['trm']))}),
    pd.DataFrame({'gb_predicted_trm': gb.predict(dollar_df.drop(columns=['trm']))})
], axis=1)


## Explicabilidad

In [ ]:
!pip install shap

In [ ]:
import shap
from IPython.display import HTML

In [ ]:
# Entrenamos nuestro pipeline con los mejores parámetros encontrados en la validación cruzada
pipeline_gb.set_params(**gb.best_params_)
pipeline_gb.fit(X_train, y_train)

In [ ]:
pipeline_gb.named_steps.keys()

In [ ]:
#Aplicamos el preproceso a los datos
X_train_preprocessed = pipeline_gb.named_steps["preprocessor"].transform(X_train)
X_test_preprocessed = pipeline_gb.named_steps["preprocessor"].transform(X_test)


In [ ]:
# Entrenemos nuestro modelo de explicabilidad
gb_explainer = shap.TreeExplainer(pipeline_gb.named_steps["regressor"])
train_gb_shap_values = gb_explainer.shap_values(X_train_preprocessed)
test_gb_shap_values = gb_explainer.shap_values(X_test_preprocessed)


In [ ]:
# Summary plot
shap.summary_plot(test_gb_shap_values, X_test, plot_type="bar")


In [ ]:
# Grafiquemos los valores SHAP para un ejemplo del conjunto de test
shap.initjs()

instance_index = 0
force_plot_html = shap.force_plot(gb_explainer.expected_value, test_gb_shap_values[instance_index], X_test.iloc[instance_index],show=False)
HTML(force_plot_html.html())


In [ ]:
# Grafiquemos la dependencia de los valores SHAP con la variable "prime_rate"
shap.dependence_plot('prime_rate', test_gb_shap_values, X_test)
